In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
import numpy as np
import pandas as pd
import hvplot.pandas
import plotly.express as px
import dataframe_image as dfi
import tensorflow as tf

In [34]:
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from imblearn.metrics import classification_report_imbalanced

In [35]:
car_clean_df = pd.read_csv("Resources/car_clean.csv")
car_clean_df.head()

,Unnamed: 0,manufacturer,model,year,selling_price,fuel,seller_type,mileage
0,0,Maruti,Swift Dzire VDI,2014,450000,Diesel,Individual,23.40
1,1,Skoda,Rapid 1.5 TDI Ambition,2014,370000,Diesel,Individual,21.14
2,2,Honda,City 2017-2020 EXi,2006,158000,Petrol,Individual,17.70
3,3,Hyundai,i20 Sportz Diesel,2010,225000,Diesel,Individual,23.00
4,4,Maruti,Swift VXI BSIII,2007,130000,Petrol,Individual,16.10


In [36]:
# Generate our categorical variable lists
df_cat = car_clean_df.dtypes[car_clean_df.dtypes == "object"].index.tolist()

car_clean_df[df_cat].nunique()

manufacturer      31
model           1983
fuel               3
seller_type        3
dtype: int64

In [37]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(car_clean_df[df_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(df_cat)
encode_df.head()

,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,manufacturer_Datsun,manufacturer_Fiat,manufacturer_Force,manufacturer_Ford,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [38]:
# Merge one-hot encoded features and drop the originals
car_clean_df = car_clean_df.merge(encode_df,left_index=True, right_index=True)
car_clean_df = car_clean_df.drop(df_cat, 1)
car_clean_df.head()

,Unnamed: 0,year,selling_price,mileage,manufacturer_Ambassador,manufacturer_Ashok,manufacturer_Audi,manufacturer_BMW,manufacturer_Chevrolet,manufacturer_Daewoo,...,model_i20 Sportz AT 1.4,model_i20 Sportz Diesel,model_i20 Sportz Option 1.2,model_i20 Sportz Option 1.4 CRDi,fuel_Diesel,fuel_Other,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer
0,0,2014,450000,23.40,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,2014,370000,21.14,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2,2006,158000,17.70,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,3,2010,225000,23.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,4,2007,130000,16.10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


### Predicting Diesel

In [39]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Diesel"].values
X = car_clean_df.drop(["fuel_Diesel"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 80)                161920    
                                                                 
 dense_10 (Dense)            (None, 30)                2430      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 164,381
Trainable params: 164,381
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [43]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 1s 2ms/step - loss: 0.3145 - accuracy: 0.8702
Epoch 2/100
186/186 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 0.9958
Epoch 3/100
186/186 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 4/100
186/186 [==============================] - 1s 3ms/step - loss: 9.2932e-04 - accuracy: 1.0000
Epoch 5/100
181/186 [============================>.] - ETA: 0s - loss: 5.3097e-04 - accuracy: 1.0000
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 1s 3ms/step - loss: 5.2509e-04 - accuracy: 1.0000
Epoch 6/100
186/186 [==============================] - 1s 3ms/step - loss: 3.3699e-04 - accuracy: 1.0000
Epoch 7/100
186/186 [==============================] - 0s 2ms/step - loss: 2.3282e-04 - accuracy: 1.0000
Epoch 8/100
186/186 [==============================] - 0s 3ms/step - loss: 1.6998e-04 - accuracy: 1.0000
Epoch 9/100
186/186 [=========

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 0s - loss: 0.0043 - accuracy: 0.9985 - 391ms/epoch - 6ms/step
Loss: 0.004310120362788439, Accuracy: 0.9984825253486633


### Predicting Other Fuel

In [45]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Other"].values
X = car_clean_df.drop(["fuel_Other"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [46]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                161920    
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 164,381
Trainable params: 164,381
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0706 - accuracy: 0.9838
Epoch 2/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0136 - accuracy: 0.9961
Epoch 3/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0042 - accuracy: 0.9993
Epoch 4/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 5/100
182/186 [============================>.] - ETA: 0s - loss: 4.9923e-04 - accuracy: 1.0000
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 0s 3ms/step - loss: 4.9201e-04 - accuracy: 1.0000
Epoch 6/100
186/186 [==============================] - 0s 2ms/step - loss: 2.5581e-04 - accuracy: 1.0000
Epoch 7/100
186/186 [==============================] - 1s 3ms/step - loss: 1.5305e-04 - accuracy: 1.0000
Epoch 8/100
186/186 [==============================] - 1s 3ms/step - loss: 1.0014e-04 - accuracy: 1.0000
Epoch 9/100
186/186 [=============

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 0s - loss: 0.0604 - accuracy: 0.9965 - 291ms/epoch - 5ms/step
Loss: 0.06040437892079353, Accuracy: 0.9964593052864075


### Predicting Petrol Fuel

In [51]:
# Split our preprocessed data into our features and target arrays
y = car_clean_df["fuel_Petrol"].values
X = car_clean_df.drop(["fuel_Petrol"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [52]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [53]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 80)                161920    
                                                                 
 dense_16 (Dense)            (None, 30)                2430      
                                                                 
 dense_17 (Dense)            (None, 1)                 31        
                                                                 
Total params: 164,381
Trainable params: 164,381
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch', 
    period=5)

In [55]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
186/186 [==============================] - 1s 3ms/step - loss: 0.3935 - accuracy: 0.8241
Epoch 2/100
186/186 [==============================] - 0s 3ms/step - loss: 0.0203 - accuracy: 0.9965
Epoch 3/100
186/186 [==============================] - 0s 2ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 4/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 5/100
175/186 [===========================>..] - ETA: 0s - loss: 6.7241e-04 - accuracy: 1.0000
Epoch 5: saving model to checkpoints\weights.05.hdf5
186/186 [==============================] - 0s 3ms/step - loss: 6.6153e-04 - accuracy: 1.0000
Epoch 6/100
186/186 [==============================] - 0s 2ms/step - loss: 4.2154e-04 - accuracy: 1.0000
Epoch 7/100
186/186 [==============================] - 1s 3ms/step - loss: 2.9127e-04 - accuracy: 1.0000
Epoch 8/100
186/186 [==============================] - 1s 3ms/step - loss: 2.1182e-04 - accuracy: 1.0000
Epoch 9/100
186/186 [=============

In [56]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

62/62 - 0s - loss: 0.0187 - accuracy: 0.9934 - 249ms/epoch - 4ms/step
Loss: 0.01873369887471199, Accuracy: 0.9934243559837341
